# DEA CoastLines vector statistics

## To do:
* [ ] Update for 2019


### Load packages

First we import the required Python packages, then we connect to the database, and load the catalog of virtual products.

In [1]:
%matplotlib inline
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

import deacoastlines_statistics as deacl_stats

import os
import sys
import geopandas as gpd
from shapely.geometry import box
from rasterio.transform import array_bounds
import pandas as pd
import shutil
import matplotlib.pyplot as plt


## Load in data

In [2]:
# Read in contours
study_area = 8032
output_name = 'v0.2.0'
water_index = 'mndwi'
index_threshold = 0.00
baseline_year = '2019'


## Load DEA CoastLines rasters

In [3]:
yearly_ds = deacl_stats.load_rasters(output_name, study_area, water_index)
print(yearly_ds)

# Create output vector folder
output_dir = f'output_data/{study_area}_{output_name}/vectors'
os.makedirs(f'{output_dir}/shapefiles', exist_ok=True)

<xarray.Dataset>
Dimensions:         (x: 1242, y: 1791, year: 32)
Coordinates:
  * y               (y) float64 -3.655e+06 -3.655e+06 ... -3.709e+06 -3.709e+06
  * x               (x) float64 3.552e+05 3.553e+05 ... 3.924e+05 3.925e+05
  * year            (year) int64 1988 1989 1990 1991 ... 2016 2017 2018 2019
Data variables:
    mndwi           (year, y, x) float32 -0.40706956 -0.36689034 ... 0.47096777
    gapfill_index   (year, y, x) float32 -0.43252143 -0.45812184 ... 0.45173746
    gapfill_tide_m  (year, y, x) float32 -0.011186913 ... 0.02794835
    gapfill_count   (year, y, x) int16 13 13 13 13 13 13 ... 32 32 32 32 32 33
    stdev           (year, y, x) float32 0.14469294 0.13621381 ... 0.26351276
    tide_m          (year, y, x) float32 0.058467366 0.058448505 ... 0.055922028
    count           (year, y, x) int16 5 5 5 5 5 5 5 5 ... 14 14 14 15 15 15 15
Attributes:
    crs:        +init=epsg:32656
    transform:  | 30.00, 0.00, 355215.00|\n| 0.00,-30.00,-3655245.00|\n| 0.00...

## Load vector data

In [4]:
# Get bounding box to load data for
bbox = gpd.GeoSeries(box(*array_bounds(height=yearly_ds.sizes['y'],
                                       width=yearly_ds.sizes['x'],
                                       transform=yearly_ds.transform)),
                     crs=yearly_ds.crs)

# Rocky shore mask
smartline_gdf = (gpd.read_file('input_data/Smartline.gdb', 
                               bbox=bbox).to_crs(yearly_ds.crs))

# Tide points
points_gdf = (gpd.read_file('input_data/tide_points_coastal.geojson', 
                            bbox=bbox).to_crs(yearly_ds.crs))

# Study area polygon
comp_gdf = (gpd.read_file('input_data/50km_albers_grid_clipped.geojson', 
                          bbox=bbox)
            .set_index('id')
            .to_crs(str(yearly_ds.crs)))

# Mask to study area
study_area_poly = comp_gdf.loc[study_area]

# Load climate indices
climate_df = pd.read_csv('input_data/climate_indices.csv', index_col='year')

## Extract shoreline contours

### Extract ocean-masked contours

In [5]:
# Generate waterbody mask
waterbody_array = deacl_stats.waterbody_mask(
    input_data='input_data/SurfaceHydrologyPolygonsRegional.gdb',
    modification_data='input_data/estuary_mask_modifications.geojson',
    bbox=bbox,
    yearly_ds=yearly_ds)

In [6]:
# Mask dataset to focus on coastal zone only
masked_ds = deacl_stats.contours_preprocess(
    yearly_ds,
    water_index,
    index_threshold,
    waterbody_array,
    points_gdf,
    output_path=f'output_data/{study_area}_{output_name}')

In [7]:
# Extract contours
contours_gdf = (deacl_stats.subpixel_contours(da=masked_ds,
                                             z_values=index_threshold,
                                             min_vertices=30,
                                             dim='year',
                                             output_path='temp.geojson')
                .set_index('year'))

Operating in single z-value, multiple arrays mode
Writing contours to temp.geojson


## Compute statistics
### Create stats points on baseline contour

In [8]:
# Extract statistics modelling points along baseline contour
points_gdf = deacl_stats.stats_points(contours_gdf, baseline_year, distance=30)

# Clip to remove rocky shoreline points
points_gdf = deacl_stats.rocky_shores_clip(points_gdf, smartline_gdf, buffer=50)


### Measure annual coastline movements

In [9]:
if points_gdf is not None:
    
    # Make a copy of the points GeoDataFrame to hold tidal data
    tide_points_gdf = points_gdf.copy()

    # Calculate annual movements and residual tide heights for every contour
    # compared to the baseline year
    points_gdf, tide_points_gdf = deacl_stats.annual_movements(yearly_ds, 
                                                               points_gdf, 
                                                               tide_points_gdf, 
                                                               contours_gdf, 
                                                               baseline_year,
                                                               water_index)

### Calculate regressions

In [11]:
if points_gdf is not None:

    points_gdf = deacl_stats.calculate_regressions(yearly_ds, 
                                                   points_gdf, 
                                                   tide_points_gdf, 
                                                   climate_df)

Comparing annual movements with time


In [12]:
points_gdf

,rate_time,sig_time,se_time,outl_time,1988,1989,1990,1991,1992,1993,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,geometry
0,0.889,0.005,0.293,,2.43,1.04,-21.66,-4.77,-4.90,5.94,...,34.87,33.91,38.00,41.14,40.02,2.45,6.12,2.26,-0.0,POINT (374970.000 -3661409.459)
1,0.451,0.038,0.207,,-4.64,-17.17,-7.38,3.55,3.99,9.81,...,19.91,17.85,15.76,15.16,17.60,7.89,-7.18,-13.71,0.0,POINT (374979.473 -3661380.994)
2,0.431,0.024,0.181,,-3.81,-8.82,-5.11,-1.39,0.19,6.88,...,18.09,15.89,12.78,12.64,18.11,7.17,-5.35,-8.85,-0.0,POINT (374959.929 -3661358.947)
3,0.456,0.011,0.169,,-4.53,-5.19,-3.74,-0.19,-3.25,4.25,...,14.60,15.48,12.04,12.86,15.86,9.62,-5.79,-7.03,-0.0,POINT (374939.828 -3661336.681)
4,0.469,0.014,0.179,,-3.78,-4.49,-0.84,2.42,-5.78,5.77,...,15.15,17.34,12.94,14.65,15.67,11.69,-5.73,-5.86,-0.0,POINT (374919.342 -3661314.766)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1815,-0.427,0.004,0.136,,32.29,31.07,33.28,36.49,30.82,28.88,...,27.81,21.97,27.66,21.94,27.55,28.66,35.21,15.36,-0.0,POINT (360767.646 -3690675.999)
1816,-0.576,0.000,0.145,,45.26,37.79,32.54,38.70,31.36,29.78,...,35.94,24.91,24.67,20.78,18.53,22.52,37.31,9.31,-0.0,POINT (360754.503 -3690651.070)
1817,-0.514,0.043,0.243,,70.91,52.44,37.04,27.41,29.84,28.24,...,46.60,41.21,28.55,32.42,19.66,21.55,49.73,15.83,-0.0,POINT (360726.757 -3690640.915)
1818,-0.162,0.516,0.246,1988,97.51,58.00,37.63,28.75,30.65,29.98,...,60.75,43.38,31.46,38.09,25.89,25.86,61.49,27.26,0.0,POINT (360697.893 -3690632.738)


## Export files

### Export stats files

In [13]:
if points_gdf is not None:
    
    col_schema = [('rate_time', 'float:8.2'), 
                  ('sig_time', 'float:8.3'),
                  ('se_time', 'float:8.2'),
                  ('outl_time', 'str:80'), 
#                   ('rate_SOI', 'float:8.2'),
#                   ('rate_IOD', 'float:8.2'), ('rate_SAM', 'float:8.2'),
#                   ('rate_IPO', 'float:8.2'), ('rate_PDO', 'float:8.2'),
#                   ('rate_tide', 'float:8.2'), 
#                   ('sig_SOI', 'float:8.3'), ('sig_IOD', 'float:8.3'),
#                   ('sig_SAM', 'float:8.3'), ('sig_IPO', 'float:8.3'),
#                   ('sig_PDO', 'float:8.3'), ('sig_tide', 'float:8.3'),
#                   ('outl_SOI', 'str:80'),
#                   ('outl_IOD', 'str:80'), ('outl_SAM', 'str:80'),
#                   ('outl_IPO', 'str:80'), ('outl_PDO', 'str:80'),
#                   ('outl_tide', 'str:80'), 
                  ('1988', 'float:8.2'),
                  ('1989', 'float:8.2'), ('1990', 'float:8.2'),
                  ('1991', 'float:8.2'), ('1992', 'float:8.2'),
                  ('1993', 'float:8.2'), ('1994', 'float:8.2'),
                  ('1995', 'float:8.2'), ('1996', 'float:8.2'),
                  ('1997', 'float:8.2'), ('1998', 'float:8.2'),
                  ('1999', 'float:8.2'), ('2000', 'float:8.2'),
                  ('2001', 'float:8.2'), ('2002', 'float:8.2'),
                  ('2003', 'float:8.2'), ('2004', 'float:8.2'),
                  ('2005', 'float:8.2'), ('2006', 'float:8.2'),
                  ('2007', 'float:8.2'), ('2008', 'float:8.2'),
                  ('2009', 'float:8.2'), ('2010', 'float:8.2'),
                  ('2011', 'float:8.2'), ('2012', 'float:8.2'),
                  ('2013', 'float:8.2'), ('2014', 'float:8.2'),
                  ('2015', 'float:8.2'), ('2016', 'float:8.2'),
                  ('2017', 'float:8.2'), ('2018', 'float:8.2'),
                  ('2019', 'float:8.2')]

    # Clip stats to study area extent, remove rocky shores
    stats_path = f'{output_dir}/stats_{study_area}_{output_name}_{water_index}_{index_threshold:.2f}'
    points_gdf = points_gdf[points_gdf.intersects(study_area_poly['geometry'])]

    # Export to GeoJSON
    points_gdf.to_crs('EPSG:4326').to_file(f'{stats_path}.geojson', 
                                           driver='GeoJSON')

    # Export as ESRI shapefiles
    stats_path = stats_path.replace('vectors', 'vectors/shapefiles')
    points_gdf.to_file(f'{stats_path}.shp',
                       schema={'properties': col_schema,
                               'geometry': 'Point'})

### Export contours

In [40]:
# Assign certainty to contours based on underlying masks
contours_gdf = deacl_stats.contour_certainty(contours_gdf, 
                                           output_path=f'output_data/{study_area}_{output_name}')

# Clip annual shoreline contours to study area extent
contour_path = f'{output_dir}/contours_{study_area}_{output_name}_{water_index}_{index_threshold:.2f}'
contours_gdf['geometry'] = contours_gdf.intersection(study_area_poly['geometry'])
contours_gdf.reset_index().to_crs('EPSG:4326').to_file(f'{contour_path}.geojson', 
                                                       driver='GeoJSON')

# Export stats and contours as ESRI shapefiles
contour_path = contour_path.replace('vectors', 'vectors/shapefiles')
contours_gdf.reset_index().to_file(f'{contour_path}.shp')

/g/data/v10/public/modules/dea-env/20200612/lib/python3.6/site-packages/geopandas/geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()


## Merge contours and rasters

In [ ]:
# Export coastlines
!ogrmerge.py -o DEACoastLines_coastlines_v0.2.0_0.10.shp output_data/*/vectors/shapefiles/contours_*_v0.2.0_mndwi_0.10.shp -single -overwrite_ds -t_srs EPSG:4326

In [ ]:
# Export statistics
!ogrmerge.py -o DEACoastLines_statistics_v0.2.0.shp output_data/*/vectors/shapefiles/stats_*_v0.2.0_mndwi_0.00.shp -single -overwrite_ds -t_srs EPSG:4326

In [ ]:
# Export mask
!gdalwarp output_data/*v0.2.0/all_time_mask.tif mask_v0.2.0.tif -co COMPRESS=DEFLATE -co ZLEVEL=9 -co PREDICTOR=1 -co TILED=YES -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 -overwrite -q

In [ ]:
!gdalwarp output_data/*v0.2.0/1988_count.tif 1988_count.tif -co COMPRESS=DEFLATE -co ZLEVEL=5 -co PREDICTOR=1 -co TILED=YES -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 -overwrite -q

In [ ]:
!gdalwarp output_data/*v0.2.0/2011_mndwi.tif 2011_mndwi.tif -co COMPRESS=DEFLATE -co ZLEVEL=5 -co PREDICTOR=1 -co TILED=YES -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 -co BIGTIFF=YES -overwrite -q

In [ ]:
!gdaladdo -r average 1988_count.tif 16

## Generate continental summary layer

In [ ]:
from deacoastlines_summary import main

In [ ]:
main(['out', 'v0.2.0'])

## Assorted

In [ ]:
def gadi_su(memory, time, queue='normal'):
    
    if queue == 'normal':
        print((memory / 4) * 2 * time)
    if queue == 'hugemem':
        print((memory / 32) * 3 * time)
        
gadi_su(memory=96, time=6, queue='hugemem')        
        

In [ ]:
import geopandas as gpd

col_schema = [('rate_time', 'float:8.2'), ('rate_SOI', 'float:8.2'),
              ('rate_IOD', 'float:8.2'), ('rate_SAM', 'float:8.2'),
              ('rate_IPO', 'float:8.2'), ('rate_PDO', 'float:8.2'),
              ('rate_tide', 'float:8.2'), ('sig_time', 'float:8.3'),
              ('sig_SOI', 'float:8.3'), ('sig_IOD', 'float:8.3'),
              ('sig_SAM', 'float:8.3'), ('sig_IPO', 'float:8.3'),
              ('sig_PDO', 'float:8.3'), ('sig_tide', 'float:8.3'),
              ('outl_time', 'str:80'), ('outl_SOI', 'str:80'),
              ('outl_IOD', 'str:80'), ('outl_SAM', 'str:80'),
              ('outl_IPO', 'str:80'), ('outl_PDO', 'str:80'),
              ('outl_tide', 'str:80'), ('1988', 'float:8.2'),
              ('1989', 'float:8.2'), ('1990', 'float:8.2'),
              ('1991', 'float:8.2'), ('1992', 'float:8.2'),
              ('1993', 'float:8.2'), ('1994', 'float:8.2'),
              ('1995', 'float:8.2'), ('1996', 'float:8.2'),
              ('1997', 'float:8.2'), ('1998', 'float:8.2'),
              ('1999', 'float:8.2'), ('2000', 'float:8.2'),
              ('2001', 'float:8.2'), ('2002', 'float:8.2'),
              ('2003', 'float:8.2'), ('2004', 'float:8.2'),
              ('2005', 'float:8.2'), ('2006', 'float:8.2'),
              ('2007', 'float:8.2'), ('2008', 'float:8.2'),
              ('2009', 'float:8.2'), ('2010', 'float:8.2'),
              ('2011', 'float:8.2'), ('2012', 'float:8.2'),
              ('2013', 'float:8.2'), ('2014', 'float:8.2'),
              ('2015', 'float:8.2'), ('2016', 'float:8.2'),
              ('2017', 'float:8.2'), ('2018', 'float:8.2')]

fname=f'DEACoastLines_statistics_v0.2.0.shp'
stats_gdf = gpd.read_file(fname)
stats_gdf.to_file(f'DEACoastLines_statistics_v0.2.0_schema.shp', 
                  schema={'properties': col_schema, 
                          'geometry': 'Point'})


In [ ]:
import shutil
shutil.make_archive('DEACoastLines_v0.2.0', 'zip', '/g/data/r78/rt1527/dea-notebooks/MAHTS/packages/DEACoastLines_v0.2.0')


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** April 2020